# Method: VCT regression

In [1]:
#import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [93]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
import numpy as np
import yaml

from vessel_manoeuvring_models.visualization.plot import track_plot, plot
from phd.visualization.plot_ship import track_plots
from vessel_manoeuvring_models.prime_system import PrimeSystem
from phd.visualization.plot_prediction import plot_total_force, plot_force_components, predict, plot_compare_model_forces, plot_parameter_contributions
from phd.visualization.bokeh_plotting import create_tab
from vessel_manoeuvring_models.symbols import *
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify, remove_functions, prime
from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]
import statsmodels.api as sm
import sympy as sp
from sympy import Eq, symbols, latex
import paper
from paper import save_fig, save_eq
import vessel_manoeuvring_models
from phd.pipelines.regression_VCT.nodes import df_VCT_to_prime, regress_VCT
from phd.pipelines.regression_VCT.regression_pipeline import pipeline, pipeline_RHI

In [29]:
#%reload_kedro
ship='wPCC'
ship_data = catalog.load(f"{ship}.ship_data")
model_loaders = catalog.load(f"{ship}.models_VCT")
#models_rudder_VCT_loaders = catalog.load(f"{ship}.models_rudder_VCT")
df_VCT = catalog.load(f"{ship}.df_VCT_scaled")
exclude_parameters = catalog.load(f"params:{ship}.VCT_exclude_parameters")

[02/07/24 11:36:45] INFO     Loading data from 'wPCC.ship_data' (YAMLDataSet)...                ]8;id=421590;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=569373;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'wPCC.models_VCT' (PartitionedDataset)...        ]8;id=378971;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=812362;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'wPCC.df_VCT_scaled' (CSVDataSet)...             ]8;id=187125;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=20915;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:wPCC.VCT_exclude_parameters'             ]8;id=518273;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=846491;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\
                             (MemoryDataset)...                                                                    

In [30]:
exclude_parameters


{
    'Y0': 0,
    'N0': 0,
    'Yvdeltadelta': 0,
    'Yvvdelta': 0,
    'Yrdeltadelta': 0,
    'Yrrdelta': 0,
    'Yvrdelta': 0,
    'Nrrdelta': 0,
    'Nvrdelta': 0,
    'Nvdeltadelta': 0,
    'Nrdeltadelta': 0,
    'Nvvdelta': 0,
    'Xthrustport': 0.883,
    'Xthruststbd': 0.883,
    'Yvvv': 0,
    'Nvvv': 0,
    'Nrrr': 0,
    'Yvvr': 0,
    'Yvrr': 0,
    'Nvvr': 0,
    'Nvrr': 0,
    'Yr': 0,
    'Yrrr': 0
}

## Mathematical rudder model

In [31]:
model = model_loaders['semiempirical_covered']()

In [32]:
model.show_subsystems()

hull: PrimeEquationSubSystem
propellers: PropellersSimpleSystem
rudder_port: SemiempiricalRudderSystemCovered
rudder_stbd: SemiempiricalRudderSystemCovered
rudders: Rudders
rudder_hull_interaction: RudderHullInteractionSystem
wind_force: DummyWindForceSystem


In [33]:
df_VCT_prime = df_VCT_to_prime(model=model, df_VCT=df_VCT)

In [34]:
regression_pipeline = pipeline(df_VCT_prime=df_VCT_prime, model=model)

In [35]:
regression_pipeline.keys()

dict_keys(['resistance', 'Drift angle fx', 'Drift angle fy', 'Drift angle mz', 'Circle fx', 'Circle fy', 'Circle mz', 'Circle + Drift fx', 'Circle + Drift fy', 'Circle + Drift mz'])

In [36]:
regression = regression_pipeline['Drift angle fy']

In [37]:
regression['eq']

Eq(Y_H, Y_{0} + Y_{vvv}*v**3 + Y_{v}*v)

In [38]:
for name,regression in regression_pipeline.items():
    display(regression['eq'])

Eq(X_H, X_{0} + X_{u}*u)

Eq(X_H, X_{0} + X_{u}*u + X_{vv}*v**2)

Eq(Y_H, Y_{0} + Y_{vvv}*v**3 + Y_{v}*v)

Eq(N_H, N_{0} + N_{vvv}*v**3 + N_{v}*v)

Eq(X_H, X_{0} + X_{rr}*r**2 + X_{u}*u)

Eq(Y_H, Y_{0} + Y_{rrr}*r**3 + Y_{r}*r)

Eq(N_H, N_{0} + N_{rrr}*r**3 + N_{r}*r)

Eq(X_H, X_{0} + X_{rr}*r**2 + X_{u}*u + X_{vr}*r*v + X_{vv}*v**2)

Eq(Y_H, Y_{0} + Y_{rrr}*r**3 + Y_{r}*r + Y_{vrr}*r**2*v + Y_{vvr}*r*v**2 + Y_{vvv}*v**3 + Y_{v}*v)

Eq(N_H, N_{0} + N_{rrr}*r**3 + N_{r}*r + N_{vrr}*r**2*v + N_{vvr}*r*v**2 + N_{vvv}*v**3 + N_{v}*v)

In [39]:
df_VCT['test type'].unique()


array(['Circle', 'Circle + Drift', 'Circle + rudder angle', 'Drift angle',
       'Rudder and drift angle', 'Rudder angle', 'Thrust variation',
       'self propulsion', 'Rudder angle resistance (no propeller)'],
      dtype=object)

In [40]:
df_VCT.groupby(by='test type').max()[['u','v','r','delta','thrust']]

,u,v,r,delta,thrust
test type,,,,,
Circle,0.961770,-0.000000,-0.023974,-0.000000,10.394889
Circle + Drift,0.961770,0.167010,0.047949,-0.000000,9.709370
Circle + rudder angle,0.961770,-0.000000,-0.023974,0.069813,9.291126
Drift angle,0.961184,-0.033565,0.000000,-0.000000,10.664554
Rudder and drift angle,0.959427,-0.067090,0.000000,0.069813,9.410107
Rudder angle,0.961770,-0.000000,0.000000,-0.052360,9.229330
Rudder angle resistance (no propeller),0.961770,-0.000000,0.000000,-0.087266,0.000000
Thrust variation,0.961770,-0.000000,0.000000,-0.174533,13.721516
self propulsion,0.961770,-0.000000,0.000000,-0.000000,9.130484


In [41]:
df_VCT.groupby(by='test type').min()[['u','v','r','delta','thrust']]

,u,v,r,delta,thrust
test type,,,,,
Circle,0.961770,-0.000000,-0.127864,-0.000000,9.249258
Circle + Drift,0.947158,-0.167010,-0.047949,-0.000000,9.095822
Circle + rudder angle,0.961770,-0.000000,-0.023974,-0.069813,9.228015
Drift angle,0.903768,-0.328945,0.000000,-0.000000,9.124859
Rudder and drift angle,0.959427,-0.067090,0.000000,-0.069813,9.381886
Rudder angle,0.961770,-0.000000,0.000000,-0.261799,9.095363
Rudder angle resistance (no propeller),0.961770,-0.000000,0.000000,-0.261799,0.000000
Thrust variation,0.480885,-0.000000,0.000000,-0.174533,1.360484
self propulsion,0.480885,-0.000000,0.000000,-0.000000,2.577043


In [42]:
eq_N_D = model.expand_subsystemequations(model.N_D_eq)

In [43]:
eq_N_D.subs([
    (v,0),
    (r,0),
    (delta,0),
    (thrust_port,0),
    (thrust_stbd,0),
])

Eq(N_D, N_R*x_H + N_{0} + x_R*(D_R_port*sin(gamma_0_port + atan(V_R_y_port/V_R_x_C_port)) + L_R_port*cos(gamma_0_port + atan(V_R_y_port/V_R_x_C_port))) + x_R*(D_R_stbd*sin(gamma_0_stbd + atan(V_R_y_stbd/V_R_x_C_stbd)) + L_R_stbd*cos(gamma_0_stbd + atan(V_R_y_stbd/V_R_x_C_stbd))))

In [44]:
exclude_parameters


{
    'Y0': 0,
    'N0': 0,
    'Yvdeltadelta': 0,
    'Yvvdelta': 0,
    'Yrdeltadelta': 0,
    'Yrrdelta': 0,
    'Yvrdelta': 0,
    'Nrrdelta': 0,
    'Nvrdelta': 0,
    'Nvdeltadelta': 0,
    'Nrdeltadelta': 0,
    'Nvvdelta': 0,
    'Xthrustport': 0.883,
    'Xthruststbd': 0.883,
    'Yvvv': 0,
    'Nvvv': 0,
    'Nrrr': 0,
    'Yvvr': 0,
    'Yvrr': 0,
    'Nvvr': 0,
    'Nvrr': 0,
    'Yr': 0,
    'Yrrr': 0
}

In [94]:
model_,models = regress_VCT(model=model, df_VCT=df_VCT, pipeline=pipeline_RHI, exclude_parameters=exclude_parameters)
model_,models_ = regress_VCT(model=model, df_VCT=df_VCT, pipeline=pipeline, exclude_parameters=exclude_parameters)
models.update(models_)

[02/07/24 15:39:29] INFO     Excluding:{'Y0': 0, 'N0': 0, 'Yvdeltadelta': 0, 'Yvvdelta': ]8;id=180741;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py\regression_pipeline.py]8;;\:]8;id=440003;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py#286\286]8;;\
                             0, 'Yrdeltadelta': 0, 'Yrrdelta': 0, 'Yvrdelta': 0,                                   
                             'Nrrdelta': 0, 'Nvrdelta': 0, 'Nvdeltadelta': 0,                                      
                             'Nrdeltadelta': 0, 'Nvvdelta': 0, 'Xthrustport': 0.883,                               
                             'Xthruststbd': 0.883, 'Yvvv': 0, 'Nvvv': 0, 'Nrrr': 0,                                
                             'Yvvr': 0, 'Yvrr': 0, 'Nvvr': 0, 'Nvrr': 0, 'Yr': 0,                                  
                             'Yrrr': 0}                                                                            

                    INFO     Fitting:rudder hull interaction aH                          ]8;id=352176;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py\regression_pipeline.py]8;;\:]8;id=296970;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py#288\288]8;;\

[02/07/24 15:39:30] INFO     Fitting:rudder hull interaction xH                          ]8;id=56952;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py\regression_pipeline.py]8;;\:]8;id=289972;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py#288\288]8;;\

                    INFO     Regression:rudder hull interaction aH                                     ]8;id=331457;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py\nodes.py]8;;\:]8;id=182427;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py#662\662]8;;\

Y_D = Y_R⋅a_H


...

                    INFO                             Results: Ordinary least squares                   ]8;id=266101;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py\nodes.py]8;;\:]8;id=275188;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py#665\665]8;;\
                             =========================================================================             
                             =======                                                                               
                             Model:                  OLS              Adj. R-squared (uncentered):                 
                             0.938                                                                                 
                             Dependent Variable:     N_H              AIC:                                         
                             -72.8094                                                                              
                             Date:                   2024-02-07 15:39 BIC:                                         
                             -73.2000                                                                              
                             No. Observations:       5                Log-Likelihood:                              
                             37.405                                                                                
                             Df Model:               1                F-statistic:                                 
                             76.80                                                                                 
                             Df Residuals:           4                Prob (F-statistic):                          
                             0.000935                                                                              
                             R-squared (uncentered): 0.950            Scale:                                       
                             2.3258e-08                                                                            
                             -------------------------------------------------------------------------             
                             -------------                                                                         
                                       Coef.        Std.Err.          t          P>|t|         [0.025              
                             0.975]                                                                                
                             -------------------------------------------------------------------------             
                             -------------                                                                         
                             Nr       -0.0014         0.0002       -8.7637       0.0009       -0.0019              
                             -0.0010                                                                               
                             -------------------------------------------------------------------------             
                             -------                                                                               
                             Omnibus:                   nan               Durbin-Watson:                           
                             0.922                                                                                 
                             Prob(Omnibus):             nan               Jarque-Bera (JB):                        
                             1.596                                                                                 
                             Skew:                      1.383             Prob(JB):                                
                             0.450                                                                        

                    INFO     Regression:Circle + Drift fx                                              ]8;id=999663;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py\nodes.py]8;;\:]8;id=530845;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py#662\662]8;;\

                      2                                  2
X_H = X_{0} + X_{rr}⋅r  + X_{u}⋅u + X_{vr}⋅r⋅v + X_{vv}⋅v 


                    INFO     None                                                                      ]8;id=368980;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py\nodes.py]8;;\:]8;id=507233;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py#663\663]8;;\

                    INFO                       Results: Ordinary least squares                         ]8;id=82481;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py\nodes.py]8;;\:]8;id=60086;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py#665\665]8;;\
                             ===================================================================                   
                             Model:              OLS              Adj. R-squared:     0.949                        
                             Dependent Variable: X_H              AIC:                -402.6519                    
                             Date:               2024-02-07 15:39 BIC:                -400.6604                    
                             No. Observations:   20               Log-Likelihood:     203.33                       
                             Df Model:           1                F-statistic:        352.6                        
                             Df Residuals:       18               Prob (F-statistic): 2.85e-13                     
                             R-squared:          0.951            Scale:              9.6150e-11                   
                             ----------------------------------------------------------------------                
                                        Coef.     Std.Err.       t       P>|t|     [0.025    0.975]                
                             ----------------------------------------------------------------------                
                             Xvr        0.0031      0.0002    18.7771    0.0000    0.0028    0.0035                
                             const      0.0000      0.0000     3.4411    0.0029    0.0000    0.0000                
                             -------------------------------------------------------------------                   
                             Omnibus:               3.677         Durbin-Watson:           2.679                   
                             Prob(Omnibus):         0.159         Jarque-Bera (JB):        1.863                   
                             Skew:                  -0.458        Prob(JB):                0.394                   
                             Kurtosis:              1.818         Condition No.:           76                      
                             ===================================================================                   
                             Notes:                                                                                
                             [1] Standard Errors assume that the covariance matrix of the errors                   
                             is correctly specified.                                                               

In [95]:
parameters = pd.Series()
for name, regression in models.items():
    mask = regression.params.values != 0
    parameters[name] = np.array(regression.params.keys())[mask]

In [96]:
parameters


rudder hull interaction aH            [aH]
rudder hull interaction xH            [xH]
resistance                        [X0, Xu]
Drift angle fx                [Xvv, const]
Drift angle fy                        [Yv]
Drift angle mz                        [Nv]
Circle fx                     [Xrr, const]
Circle mz                             [Nr]
Circle + Drift fx             [Xvr, const]
dtype: object

In [97]:
regression.eq

Eq(X_H, X_{0} + X_{rr}*r**2 + X_{u}*u + X_{vr}*r*v + X_{vv}*v**2)

In [98]:
eq_latex = sp.latex(regression.eq.lhs)
eq_latex_prime = f"{{{eq_latex}}}'"
eq_latex_prime

"{X_{H}}'"

In [99]:
def to_latex_prime(expression):
    eq_latex = sp.latex(expression)
    eq_latex_prime = f"${{{eq_latex}}}'$"
    eq_latex_prime = eq_latex_prime.replace('delta',r'\delta')
    return eq_latex_prime

In [100]:
name_replace={
    "rudder hull interaction aH":"Rudder angle",
    "rudder hull interaction xH":"Rudder angle",
    "resistance":"Self propulsion",
}

In [108]:
variable_names = {symbol:key for key,symbol in df_parameters['symbol'].items()}
variable_names[a_H] = 'a_H'
variable_names[x_H] = 'x_H'

In [109]:
table_regression_pipeline = pd.DataFrame()
found = set([u,v,r,delta,thrust_port,thrust_stbd,p.Xthruststbd,p.Xthrustport,y_p_port, y_p_stbd,p.Y0,p.N0,Y_R,N_R])
for name, regression in models.items():
    new_parameters = regression.eq.rhs.free_symbols - found

    zeros = []
    for parameter in new_parameters:
        if model.parameters[variable_names[parameter]] == 0:
            zeros.append(parameter)

    new_parameters = new_parameters - set(zeros)
    
    found=found.union(new_parameters)
    
    test_type = str(name)
    test_type = test_type.replace(" fx","")
    test_type = test_type.replace(" fy","")
    test_type = test_type.replace(" mz","")
    table_regression_pipeline.loc[name,'Test type'] = name_replace.get(test_type,test_type) 
    table_regression_pipeline.loc[name,'Label'] = to_latex_prime(regression.eq.lhs)
    table_regression_pipeline.loc[name,'Features'] = " ".join([to_latex_prime(parameter) for parameter in new_parameters])

In [110]:
model.parameters[variable_names[p.Nvvv]] == 0

True

In [111]:
table_regression_pipeline

,Test type,Label,Features
rudder hull interaction aH,Rudder angle,${Y_{D}}'$,${a_{H}}'$
rudder hull interaction xH,Rudder angle,${N_{D}}'$,${x_{H}}'$
resistance,Self propulsion,${X_{H}}'$,${X_{u}}'$ ${X_{0}}'$
Drift angle fx,Drift angle,${X_{H}}'$,${X_{vv}}'$
Drift angle fy,Drift angle,${Y_{H}}'$,${Y_{v}}'$
Drift angle mz,Drift angle,${N_{H}}'$,${N_{v}}'$
Circle fx,Circle,${X_{H}}'$,${X_{rr}}'$
Circle mz,Circle,${N_{H}}'$,${N_{r}}'$
Circle + Drift fx,Circle + Drift,${X_{H}}'$,${X_{vr}}'$


In [112]:
table_regression_pipeline.to_csv(paper.file_path_with_nb_ref("regression_pipeline_semiempirical.csv", directory='tables'), index=False)